In [1]:
import pytest
import torch
import matplotlib.pyplot as plt

from flecs.cell_population import TestCellPop
from flecs.mutation import BernoulliMutation, GaussianMutation
from flecs.trajectory import simulate_deterministic_trajectory
from flecs.utils import plot_trajectory, set_seed

def test_mutations():
    my_cells = TestCellPop(n_cells=3)
    set_seed(0)

    # apply_bernoulli_mutation(cell_pop["gene"], "alpha", p=0.5, n_cells=3)

    # apply_gaussian_mutation(cell_pop['gene', 'activation', 'gene'], "weights", sigma=1., n_cells=3)

    # Simulate trajectory
    cell_traj = simulate_deterministic_trajectory(cell_pop, torch.linspace(0, 1, 100))

    plot_trajectory(cell_traj[:, 0], legend=False)
    plt.show()

    plot_trajectory(cell_traj[:, 1], legend=False)
    plt.show()

    plot_trajectory(cell_traj[:, 2], legend=False)
    plt.show()


ImportError: cannot import name 'BernoulliMutation' from 'flecs.mutation' (/home/jdv/code/flecs/flecs/mutation.py)